In [65]:
import numpy as np, pandas as pd, requests as rq
import json, re, time, datetime

from statsmodels.tsa.arima.model import ARIMA

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from xgboost import XGBRegressor, XGBRFRegressor

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, SimpleRNN, Conv1D, Flatten
from tensorflow.keras.models import load_model, save_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import (mean_absolute_error,
                             mean_absolute_percentage_error,
                             mean_squared_error, r2_score)

import warnings
warnings.filterwarnings("ignore")

In [37]:
def update_data(tickers = []):
    all_tickers_df = pd.DataFrame()
    for i in tickers:
        headers = {"Accept":"text/html", "Accept-Language":"en-US", "Referer":"https://www.nasdaq.com/", "User-Agent":"Chrome/64.0.3282.119"}
        resp = rq.get(f'https://api.nasdaq.com/api/quote/{i}/chart?assetclass=stocks&fromdate=2013-02-02&todate={datetime.datetime.now().strftime("%Y-%m-%d")}', headers=headers, verify=True)
        if resp.status_code == 200:
            try:
                parsed_resp = json.loads(re.search('\[.*\]', resp.text).group())
                current_ticker_df = pd.DataFrame([parsed_resp[k]['z'] for k in range(len(parsed_resp))])
                current_ticker_df['ticker'] = i
                for col_name in ['high','low','open','close','volume','value']:
                    current_ticker_df[col_name] = pd.to_numeric(current_ticker_df[col_name].str.replace(',',''))
                current_ticker_df['dateTime'] = pd.to_datetime(current_ticker_df['dateTime'])
                all_tickers_df = pd.concat([all_tickers_df, current_ticker_df])
            except KeyError:
                pass
        else:
            print(f'ERROR: smth is wrong with {i}')
    all_tickers_df.to_parquet(f'data.parquet.gz', compression='gzip')
    print(f'Info about {len(tickers)} tickers successfully loaded.')

In [38]:
class Scaler:
    __minimum = 0
    __maximum = 0

    def __init__(self, params = None):
        if params is not None:
            self.__minimum, self.__maximum = params[0], params[1]

    def fit(self, array):
        self.__minimum = np.min(array)
        self.__maximum = np.max(array)
        return self

    def scale(self, array) -> np.array:
        return (array-self.__minimum)/(self.__maximum - self.__minimum)

    def unscale(self, array) -> np.array:
        return array*(self.__maximum - self.__minimum) + self.__minimum

    def params(self) -> list:
        return self.__minimum, self.__maximum

In [39]:
def get_data(ticker = 'AAPL', column = 'close', length = 15, step = 1, start_date = '01-01-2013', split_date = '01-01-2022', end_date = '01-01-2023', flatten = False, validate = False):

    data = pd.read_parquet('data.parquet.gz')
    data = data[(data.dateTime >= start_date) & (data.ticker == ticker)]

    train_data = data[(data.dateTime < split_date)][column].values.reshape(-1, 1)
    test_data =  data[(data.dateTime >= split_date) & (data.dateTime < end_date)][column].values.reshape(-1, 1)

    train_set = [train_data[j:j+length,0] for j in range(0, len(train_data)-length, step)]
    test_set =  [test_data[j:j+length,0] for j in range(0, len(test_data)-length, step)]

    scaler = Scaler().fit(train_set)
    scaled_train_set = scaler.scale(train_set)
    scaled_test_set = scaler.scale(test_set)

    scaled_x_train, scaled_x_test = scaled_train_set[:,:length-1], scaled_test_set[:,:length-1]
    scaled_y_train, scaled_y_test = scaled_train_set[:,length-1],  scaled_test_set[:,length-1]

    if not flatten:
        scaled_x_train = np.reshape(scaled_x_train, (scaled_x_train.shape[0], length - 1,1))
        scaled_x_test = np.reshape(scaled_x_test, (scaled_x_test.shape[0], length - 1,1))

    if validate:
        val_data = data[(data.dateTime >= end_date) & (data.ticker == ticker)][column].values.reshape(-1, 1)
        val_set = [val_data[j:j+length,0] for j in range(0, len(val_data)-length, step)]
        scaled_val_set = scaler.scale(val_set)
        scaled_x_val, scaled_y_val = scaled_val_set[:,:length-1], scaled_val_set[:,length-1]
        return scaled_x_train, scaled_x_test, scaled_x_val, scaled_y_train, scaled_y_test, scaled_y_val, scaler
    else:
        return scaled_x_train, scaled_x_test, scaled_y_train, scaled_y_test, scaler

In [40]:
def get_topologies(lag = 30):
    
    neurons_per_layer = lag-1
    input_shape = (neurons_per_layer, 1)

    return {'CNN + LSTM': [Conv1D(filters = 128, kernel_size = 3, activation = 'relu', input_shape = input_shape),
                                LSTM(units = neurons_per_layer),
                                Dense(units = 1)],
            'LSTM x3': [LSTM(units = neurons_per_layer,
                            return_sequences = True,
                            input_shape = input_shape),
                        LSTM(units = neurons_per_layer,
                            return_sequences = True),
                        LSTM(units = neurons_per_layer),
                        Dense(units = 1)],
            'LSTM x2': [LSTM(units = neurons_per_layer,
                            return_sequences = True,
                            input_shape = input_shape),
                        LSTM(units = neurons_per_layer),
                        Dense(units = 1)],
            'LSTM x1': [LSTM(units = neurons_per_layer,
                            input_shape = input_shape),
                        Dense(units = 1)],
            'CNN + GRU': [Conv1D(filters = 128, kernel_size = 3, activation = 'relu', input_shape = input_shape),
                        GRU(units = neurons_per_layer),
                        Dense(units = 1)],
            'GRU x3' : [GRU(units = neurons_per_layer,
                            return_sequences = True,
                            input_shape = input_shape),
                        GRU(units = neurons_per_layer,
                            return_sequences = True),
                        GRU(units = neurons_per_layer),
                        Dense(units = 1)],
            'GRU x2' : [GRU(units = neurons_per_layer,
                            return_sequences = True,
                            input_shape = input_shape),
                        GRU(units = neurons_per_layer),
                        Dense(units = 1)],
            'GRU x1' : [GRU(units = neurons_per_layer,
                            input_shape = input_shape),
                        Dense(units = 1)],
            'CNN + SimpleRNN': [Conv1D(filters = 128, kernel_size = 3, activation = 'relu', input_shape = input_shape),
                                SimpleRNN(units = neurons_per_layer),
                                Dense(units = 1)],
            'SimpleRNN x3':[SimpleRNN(units = neurons_per_layer,
                                    return_sequences = True,
                                    input_shape = input_shape),
                            SimpleRNN(units = neurons_per_layer,
                                    return_sequences = True),
                            SimpleRNN(units = neurons_per_layer),
                            Dense(units = 1)],
            'SimpleRNN x2':[SimpleRNN(units = neurons_per_layer,
                                    return_sequences = True,
                                    input_shape = input_shape),
                            SimpleRNN(units = neurons_per_layer),
                            Dense(units = 1)],
            'SimpleRNN x1':[SimpleRNN(units = neurons_per_layer,
                                    input_shape = input_shape),
                            Dense(units = 1)],
            'CNN': [Conv1D(filters = 32, kernel_size = 5, input_shape = input_shape, activation = 'relu'),
                    Flatten(),
                    Dense(units = 1)],
            'MLP(3)': [Dense(units = neurons_per_layer, input_shape = (neurons_per_layer,)),
                    Dense(units = neurons_per_layer*2),
                    Dense(units = neurons_per_layer),
                    Dense(units = 1)],
            'MLP(2)': [Dense(units = neurons_per_layer, input_shape = (neurons_per_layer,)),
                    Dense(units = neurons_per_layer),
                    Dense(units = 1)],
            'MLP(1)': [Dense(units = neurons_per_layer, input_shape = (neurons_per_layer,)),
                    Dense(units = 1)]}

In [58]:
def evaluate_ml_models(x_train, x_test, y_train, y_test) -> list:

    statistics = []
    models = {  'LR' :  LinearRegression(),
                'DTR' : DecisionTreeRegressor(min_samples_leaf=5),
                'RFR' : RandomForestRegressor(),
                'GBR' : GradientBoostingRegressor(),
                'SVR' : SVR(kernel='linear', epsilon=1e-3),
                'CBR' : CatBoostRegressor(loss_function='MAPE'),
                'XGBR': XGBRegressor(objective='reg:squarederror'),
                'XGBRFR': XGBRFRegressor(objective = 'reg:squarederror')}
    
    for model_decription, model in models.items():
        start_time = time.time()
        model.fit(x_train, y_train)
        utilized = time.time() - start_time
        preds = model.predict(x_test)
        statistics.append({'time':utilized, 'mse': mean_squared_error(y_test, preds),'mae': mean_absolute_error(y_test, preds),'mape': mean_absolute_percentage_error(y_test, preds),'model': model_decription})
    return statistics

def evaluate_autoregressive_models(x_train, x_test, y_train, y_test) -> list:

    statistics = []
    p, q = 2, 1
    for d in range(2):
        mse, mae, mape = 0, 0, 0
        for i in range(int(len(y_test)/10)):
            start_time = time.time()
            model = ARIMA(x_test[i], order = (p, d, q), enforce_stationarity=True)
            forecast = model.fit().forecast(steps = 1)
            utilized = time.time() - start_time
            mse += (forecast - y_test[i]) **2
            mae += abs(forecast - y_test[i])
            mape += abs(forecast - y_test[i])/y_test[i]
        statistics.append({'time' : utilized, 'mse': (mse / int(len(y_test) / 10))[0],'mae': (mae / int(len(y_test) / 10))[0], 'model': f'ARIMA({p},{d},{q})' if d > 0 else f'ARMA({p},{q})'})
    return statistics

def evaluate_neural_networks(topologies_dict: dict, x_train, x_test, y_train, y_test, epochs = 5, batch_size = 1000) -> list:
    statistics = []
    for topology_description, topology in topologies_dict.items():
        current_topology_model = Sequential(topology)
        current_topology_model.compile(optimizer = 'Adamax', loss = 'mse', metrics = ['mae', 'mape'])
        start_time = time.time()
        current_topology_model.fit(x = x_train, y = y_train, epochs = epochs, batch_size = batch_size, verbose = 0)
        utilized_time = time.time() - start_time
        mse, mae, mape = current_topology_model.evaluate(x_test, y_test, verbose = 0)
        statistics.append({'time': utilized_time, 'mse': mse, 'mae': mae, 'mape': mape, 'model': topology_description})
    return statistics

In [99]:
len(get_topologies(15))

16

## Data loading
Использовались данные о стоимости акций 14 крупнейших компаний, представленных на бирже Nasdaq.  
Подготовка данных включает нарезку данных на отрезки определенной длины с помощью скользящего окна и MinMax нормализацию.

In [44]:
update_data(tickers)

Info about 14 tickers successfully loaded.


## Models benchmarking
Для определения наиболее подходящей модели, было проведено тестирование:
данные усреднялись по 13 акциям, метрики MAE/MAPE.

In [64]:
results = []
for i in tickers:

    x_train, x_test, y_train, y_test, scaler = get_data(i, flatten = True)
    results += evaluate_autoregressive_models(x_train, x_test, y_train, y_test)
    results += evaluate_ml_models(x_train, x_test, y_train, y_test)

    x_train, x_test, y_train, y_test, scaler = get_data(i)
    results += evaluate_neural_networks(get_topologies(15), x_train, x_test, y_train, y_test)
    
results = pd.DataFrame(results).groupby(by='model').mean().sort_values(by='mape')
print(f'Best model: {results.index[0]}')

Best model: GRU x2


## Hyperparameter optimization
Для подбора оптимального набора гиперпараметров использовал gridsearch, всего 1008 комбинаций.
6 вариантов числа эпох, 6 вариантов размера батча, 7 оптимизаторов и 4 функции ошибки.

In [33]:
results = []
x_train,x_test,y_train,y_test,scaler = get_data()

for ep in [2,5,10,15,25,50]:
    for bs in [32,64,128,256,512,1024]:
        for opt in ['adam', 'adamax', 'adadelta', 'adagrad', 'nadam', 'rmsprop', 'sgd']:
            for loss in ['mse', 'mae', 'mape', 'msle']:
                current_topology_model = Sequential(get_topologies(15)['GRU x_test'])
                current_topology_model.compile(optimizer = opt, loss = loss)
                current_topology_model.fit(x = x_train, y = y_train, epochs = ep, batch_size = bs, verbose = 0)
                preds = current_topology_model.predict(x_test, verbose=1).reshape(-1)
                results +=[{'mse': mean_squared_error(scaler.unscale(y_test), scaler.unscale(preds)),
                            'mae': mean_absolute_error(scaler.unscale(y_test), scaler.unscale(preds)),
                            'mape': mean_absolute_percentage_error(scaler.unscale(y_test), scaler.unscale(preds)),
                            'r2':  r2_score(scaler.unscale(y_test), scaler.unscale(preds)),
                            'epochs':ep,
                            'batch_size':bs,
                            'optimizer':opt,
                            'loss':loss}]

results = pd.DataFrame(results).sort_values(by = 'mape')
print(f'Best hyperparams combo: {results.iloc[0,:].values[-4:]}')

Best hyperparams combo: [5 32 'Nadam' 'MSE']


## Final model training
Обучал модель с учетом предыдущих результатов тестов.  
Топология: входной слой на 14 нейронов, два слоя GRU по 14 нейронов, выходной слой на 1 нейрон.  
Гиперпараметры: 5 эпох, размер батча 32, оптимизатор Nadam, функция ошибки MSE.

In [85]:
final_model = Sequential(get_topologies(15)['GRU x2'])
final_model.compile(optimizer = 'Nadam', loss = 'mse')
for i in tickers:
    try:

        x_train, x_test, y_train, y_test, scaler = get_data(ticker = i)
        final_model.fit(x = x_train, y = y_train, epochs = 5, batch_size = 32, verbose = 0)
    except:
        pass
save_model(final_model, 'trained.h5')

## Testing
Модель тестировалась на данных для обучения, данных для тестирования и данных о стоимости акций 50 других компаний.

На данных для обучения:

In [94]:
results = []
for i in tickers: 
    x_train, x_test, y_train, y_test, scaler = get_data(ticker = i)
    preds = final_model.predict(x_train, verbose=0)
    results +=[{'mse': mean_squared_error(scaler.unscale(y_train), scaler.unscale(preds)),
                'mae': mean_absolute_error(scaler.unscale(y_train), scaler.unscale(preds)),
                'mape': mean_absolute_percentage_error(scaler.unscale(y_train), scaler.unscale(preds)),
                'r2':  r2_score(scaler.unscale(y_train), scaler.unscale(preds))}]
pd.DataFrame(results).mean()

mse     15.328833
mae      2.089029
mape     0.019872
r2       0.997816
dtype: float64

На данных для тестирования:

In [98]:
results = []
for i in tickers: 
    x_train, x_test, y_train, y_test, scaler = get_data(ticker = i)
    preds = final_model.predict(x_test, verbose=0)
    results +=[{'mse': mean_squared_error(scaler.unscale(y_test), scaler.unscale(preds)),
                'mae': mean_absolute_error(scaler.unscale(y_test), scaler.unscale(preds)),
                'mape': mean_absolute_percentage_error(scaler.unscale(y_test), scaler.unscale(preds)),
                'r2':  r2_score(scaler.unscale(y_test), scaler.unscale(preds))}]
pd.DataFrame(results).mean()

mse     30.217507
mae      3.195481
mape     0.018640
r2       0.937693
dtype: float64

На данных о стоимости акций других компаний:

In [77]:
update_data(json.loads(open('ticker_test.json').read()))
tickers_for_test = json.loads(open('ticker_test.json').read())

Info about 50 tickers successfully loaded.


In [92]:
results = []
for i in tickers_for_test: 
    x_train, x_test, y_train, y_test, scaler = get_data(ticker = i)
    preds = final_model.predict(x_test, verbose=0)
    results +=[{'mse': mean_squared_error(scaler.unscale(y_test), scaler.unscale(preds)),
                'mae': mean_absolute_error(scaler.unscale(y_test), scaler.unscale(preds)),
                'mape': mean_absolute_percentage_error(scaler.unscale(y_test), scaler.unscale(preds)),
                'r2':  r2_score(scaler.unscale(y_test), scaler.unscale(preds))}]
pd.DataFrame(results).mean()

mse     27.044111
mae      3.174334
mape     0.023164
r2       0.918939
dtype: float64